In [ ]:
pip install twilio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.9 MB/s eta 0:00:00


In [ ]:
import cv2
import numpy as np
from PIL import Image as PILImage
from IPython.display import display, Javascript, Audio
from google.colab.output import eval_js
from base64 import b64decode
import wave
import time
from twilio.rest import Client

In [ ]:
# Install required libraries if not already installed
!pip install opencv-python-headless twilio

In [ ]:
# Download YOLOv4 configuration and weights
!wget https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov4.cfg -O yolov4.cfg
!wget https://github.com/AlexeyAB/darknet/releases/download/yolov4/yolov4.weights -O yolov4.weights
!wget https://raw.githubusercontent.com/AlexeyAB/darknet/master/data/coco.names -O coco.names

--2025-03-01 03:22:48--  https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov4.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12231 (12K) [text/plain]
Saving to: ‘yolov4.cfg’

yolov4.cfg          100%[===================>]  11.94K  --.-KB/s    in 0s      

2025-03-01 03:22:48 (24.1 MB/s) - ‘yolov4.cfg’ saved [12231/12231]

--2025-03-01 03:22:48--  https://github.com/AlexeyAB/darknet/releases/download/yolov4/yolov4.weights
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/75388965/749e43d0-8605-436f-b26c-12ee01c2a265?X-Amz-Algorithm=AWS

In [12]:
# Load YOLOv4 model
net = cv2.dnn.readNet("yolov4.weights", "yolov4.cfg")

# Load COCO class names
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Define vehicle and person classes
vehicle_classes = {"car", "bus", "truck", "motorcycle"}
person_classes = {"person"}
target_classes = vehicle_classes | person_classes  # Union of vehicle and person classes

# Get the output layer names
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]

# Function to capture an image from webcam (only works in Colab)
def capture_image(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function captureImage(quality) {
            const video = document.createElement('video');
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            document.body.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Wait for the video to load
            await new Promise(resolve => setTimeout(resolve, 1000));

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);

            const dataUrl = canvas.toDataURL('image/jpeg', quality);
            stream.getVideoTracks()[0].stop();
            video.remove();
            canvas.remove();

            return dataUrl;
        }
    ''')

    display(js)
    data_url = eval_js('captureImage({})'.format(quality))
    binary = b64decode(data_url.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

# Function to detect objects (vehicles and persons) in the image
def detect_objects(image):
    height, width, channels = image.shape

    # Prepare the image for the neural network
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Initialization
    class_ids, confidences, boxes = [], [], []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            # Ensure the class_id is valid and check if it's a vehicle or person
            if confidence > 0.5 and 0 <= class_id < len(classes) and classes[class_id] in target_classes:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Draw bounding boxes and labels
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(image, f"{label}: {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return image, boxes, class_ids

# Function to detect potential accidents based on proximity
def detect_accidents(boxes, class_ids):
    accident_detected = False
    for i in range(len(boxes)):
        for j in range(i + 1, len(boxes)):
            box1 = boxes[i]
            box2 = boxes[j]
            label1 = classes[class_ids[i]]
            label2 = classes[class_ids[j]]

            # Check if one of them is a vehicle and the other is a person
            if (label1 in vehicle_classes and label2 in person_classes) or (label1 in person_classes and label2 in vehicle_classes):
                # Check for overlap or close proximity
                if (box1[0] < box2[0] + box2[2] and box1[0] + box1[2] > box2[0] and
                    box1[1] < box2[1] + box2[3] and box1[1] + box1[3] > box2[1]):
                    print(f"ALERT: Potential accident detected ")
                    accident_detected = True
                    break
        if accident_detected:
            break
    if not accident_detected:
      print("No accident detected")
    return accident_detected

# Function to create and play an alarm sound
def play_alarm():
    # Parameters for the sine wave
    frequency = 3000  # Hz
    fs = 44100  # Sampling rate
    duration = 2  # Duration in seconds

    # Generate time points
    t = np.linspace(0, duration, int(fs * duration), endpoint=False)

    # Generate a sine wave at the desired frequency
    note = np.sin(4 * np.pi * frequency * t)

    # Normalize to 16-bit range
    audio = np.int16(note / np.max(np.abs(note)) * 32767)

    # Save the sine wave as a WAV file
    with wave.open('alarm.wav', 'w') as wf:
        wf.setnchannels(1)  # mono
        wf.setsampwidth(2)  # 16-bit
        wf.setframerate(fs)
        wf.writeframes(audio.tobytes())

    # Play the audio
    display(Audio('alarm.wav', autoplay=True))
# Function to make a phone call
'''def make_phone_call():
    # Twilio credentials (replace with your actual credentials)
    account_sid = 'ACa31c7c9da1fdf22a02669a5bc2e8c875'  # Replace with your Account SID
    auth_token = 'a40e1168e32f6c9aaea3655e14169c1d'  # Replace with your Auth Token
    client = Client(account_sid, auth_token)

    # Your Twilio phone number
    twilio_number = '+17623830847'  # Replace with your Twilio phone number

    # The number to call (replace with the actual phone number)
    to_number = '+917901463339'  # Replace with the recipient's phone number

    call = client.calls.create(
        twiml='<Response><Say>An accident has been detected. Please respond accordingly.</Say></Response>',
        to=to_number,
        from_=twilio_number
    )

    print(f"Call initiated, SID: {call.sid}")'''

# Function to display and process live feed automatically
def live_feed():
    try:
        while True:
            # Capture an image
            filename = capture_image()
            print(f"Captured Image: {filename}")

            # Load the captured image
            image = cv2.imread(filename)

            if image is None:
                print("Error: Image not loaded correctly.")
                continue

            # Detect vehicles and persons in the captured image
            result_image, boxes, class_ids = detect_objects(image)

            # Detect potential accidents
            accident_detected = detect_accidents(boxes, class_ids)

            # Display the output image
            result_pil_image = PILImage.fromarray(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
            display(result_pil_image)

            # Play alarm sound and make phone call if an accident is detected
            if accident_detected:
                play_alarm()
                '''make_phone_call()'''
            else:
                print("No accident detected")

            time.sleep(2)  # Add a delay to control the frame rate
    except KeyboardInterrupt:
        print("Live feed stopped by user.")

# Start live feed
live_feed()

<IPython.core.display.Javascript object>

Captured Image: photo.jpg
Live feed stopped by user.
